In [8]:
import os

# Config parameters
root = "/home/ali/graham-akhanf/EpLink/Eplink"
local_root = "/home/ali/Workspace/EpLink/Eplink"

atlas = "none"
atlases_aliases = ["Vertex", "Desikan", "Glasser 2016", "Schaefer 2018", "Yan 2023"]
atlases_path = os.path.join(local_root,"ISC-pipeline","resources","atlases_fsLR_32K",'cleaned_versions')

dataset = "eplink-p3"
task = "movie"
resampled = "N"
fwhms= [0, 5, 10]
confounds_idx = 1

results_path = os.path.join(root,"ISC-pipeline","results",f"{dataset}", "pwISC")
file_pattern = os.path.join(results_path,f"pwISC_task-{task}_hemi-{{hemi}}_fwhm-{{fwhm}}_confounds-{confounds_idx}_resampled-{resampled}_atlas-{atlas}.h5")
#bootstrap_file_pattern = "pwISC_fwhm-{fwhm}_confounds-{confounds_idx}_atlas-{atlas}_SWB-pvalues(fdr).h5"


# Participants to exclude
subjid_exclude = [68, 75, 76, 79, 80, 84, 86, 90, 91, 94, 5089, 5092, 5105,
                  7, 9, 10, 12, 18, 21, 22, 32, 35, 38, 5210, 5212, 5217, 5218, 5219, 5222, 5223, 5225, 5226, 5227]

In [5]:
import numpy as np
import scipy as sp
import h5py
import pandas as pd
import errno
import glob
import importlib
import nibabel as nib
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from scipy.stats import false_discovery_control as fdr

import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, local_root)
sys.path.insert(2, os.path.join(local_root, "ISC-pipeline"))

from utils.Surf import roi2gii, vertex2gii
import utils.Vis
importlib.reload(utils.Vis)
from utils.Vis import plot_maps

# Subject-wise Bootstrapping (SWB)

In [6]:
from sklearn.utils import check_random_state

from workflow.scripts.utils import recon_lt_matrix, vectorize_lt_matrix

def bootstrap_pw_matrix(pw_matrix, random_seed=None):
    '''This function shuffles subjects within a similarity matrix based on recommendation by Chen et al., 2016'''
    random_state = check_random_state(random_seed)
    
    n_sub = pw_matrix.shape[0]
    bootstrap_subject = sorted(random_state.choice(np.arange(n_sub), size=n_sub, replace=True))
    return pw_matrix[bootstrap_subject, :][:, bootstrap_subject]

def _load_pwISC(fpath):
    decode = lambda b: b.decode("utf-8")
    decode_np = np.vectorize(decode)

    if not os.path.isfile(fpath):
        raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), fpath)

    with h5py.File(fpath, 'r') as f:
        # Load the pairwise ISCs
        pw_ISC_raw = f['pw_ISC'][:]

        # Check if it's saved as a vector or matrix
        if len(pw_ISC_raw.shape) <= 2:
            # Convert the vectors to matrix
            n_units = pw_ISC_raw.shape[0]
            pw_ISC = []
            for u in range(n_units):
                pw_ISC.append(recon_lt_matrix(pw_ISC_raw[u]))
            pw_ISC = np.stack(pw_ISC)
        else:
            pw_ISC = pw_ISC_raw
        # Load subjects associated with the matrix
        subjects = decode_np(f['subjects'][:])
    return pw_ISC, subjects

def load_pwISC(fpath):
    ISC = []
    for h in ['L', 'R']:
        isc, subjects = _load_pwISC(fpath.format(hemi=h))
        ISC.append(isc)
    
    return np.concatenate(ISC, axis=0), subjects


In [9]:
n_perm = 10000
output_path = './eplink-p3/stats/'

for fwhm in fwhms:
    pw_ISC, subjects = load_pwISC(file_pattern.format(fwhm=fwhm,hemi='{hemi}'))
    # filter the controls subjects
    subjects = np.array(list(map(int, subjects)))
    controls = subjects>5000
    controls[np.isin(subjects, subjid_exclude)] = False

    pw_ISC = pw_ISC[:,controls,:][:,:,controls]
    pw_ISC[np.isnan(pw_ISC)] = 0

    n_unit, n_sub, _ = pw_ISC.shape

    # SWB Permutations -> save the p-values(adj.)
    stat = np.zeros((n_unit,n_perm))
    p_val = np.zeros((n_unit,1))
    for u in tqdm(range(n_unit)):
        for p in range(n_perm):
            b = bootstrap_pw_matrix(pw_ISC[u])
            stat[u,p] = np.median(b)

    pval = (stat<0).mean(axis=1)
    pval_adj = fdr(pval, method='by')

    # Save options
    output_name = f"pwISC_fwhm-{fwhm}_confounds-{confounds_idx}_atlas-{atlas}_SWB-pvalues(fdr).h5"
    output_fullpath = os.path.join(output_path, output_name)

    # Save pvalues to a HDF5 file
    with h5py.File(output_fullpath, 'w') as f:
        f.create_dataset('pval', data=pval)
        f.create_dataset('pval_adj', data=pval_adj)
        f.create_dataset('controls', data=controls)


100%|██████████| 64984/64984 [4:01:15<00:00,  4.49it/s]  


In [ ]:
bootstrap_file_pattern = f"pwISC_fwhm-{fwhm}_confounds-{confounds_idx}_atlas-{{atlas}}_SWB-pvalues(fdr).h5"
bootstrap_path =  './eplink-p3/stats/'

pwISC =
sig = dict()

for a, atlas in enumerate(atlases[1:]):
    print(f'Loading atlas: {atlas}')
    # Load ISCs
    print('Loading ISCs ...')
    _pw_ISC, subjects = load_pwISC(file_pattern.format(atlas=atlas,hemi='{hemi}'))
    # Load pvalues
    print('Loading pvalues ...')
    fpath = os.path.join(bootstrap_path, bootstrap_file_pattern).format(atlas=atlas)
    with h5py.File(fpath, 'r') as f:
        pval_adj = f['pval_adj'][:]
    print('done!')

    # Set the significance threshold
    sig[atlas] = pval_adj < 0.001  

    # Patients and healthy controls indices
    subjects = np.array(list(map(int, subjects)))
    controls = subjects>5000
    controls[np.isin(subjects, subjid_exclude)] = False
    patients = subjects<5000
    patients[np.isin(subjects, subjid_exclude)] = False

    _pw_ISC[np.isnan(_pw_ISC)] = 0

    # quick fix for the Desikan problem
    if atlas == 'Desikan':
        n_sub = _pw_ISC.shape[1]
        # sig = np.concatenate(([False], sig[:35], [False], sig[35:]))
        # pw_ISC = np.concatenate((np.zeros((1,n_sub,n_sub)), pw_ISC[:35], np.zeros((1,n_sub,n_sub)), pw_ISC[35:]))
        sig[atlas] = np.concatenate((sig[atlas][:4], [False], sig[atlas][4:39], [False], sig[atlas][39:]))
        _pw_ISC = np.concatenate((_pw_ISC[:4], np.zeros((1,n_sub,n_sub)), _pw_ISC[4:39], np.zeros((1,n_sub,n_sub)), _pw_ISC[39:]))

    pw_ISC_cont = _pw_ISC[:,controls,:][:,:,controls]
    pw_ISC_pat = _pw_ISC[:,patients,:][:,:,patients]

    # Generate the surface maps
    con_vec = np.vstack([vectorize_lt_matrix(pw_ISC_cont[u,:,:]) for u in range(pw_ISC_cont.shape[0])])
    con_vec_med = np.median(con_vec, axis=1)

    n_units = sig[atlas].shape[0]
    sig[atlas][0] = False
    sig[atlas][n_units//2] = False

    con_vec_med[~sig[atlas]] = np.nan # mask the non-significant rois 

    output_path = os.path.join('..','surface_maps',dataset)
    if atlas == 'none':
        n_units = con_vec_med.shape[0]
        con_vec_med = {'L': con_vec_med[:n_units//2], 'R': con_vec_med[n_units//2:]}
        vertex2gii(con_vec_med, output_path, f'pwISC_desc-median_fwhm-{fwhm}_confounds-{confounds_idx}_atlas-{atlas}')
    else:
        atlasfile = os.path.join(atlases_path,f'{atlas}.32k.{{hemi}}.label.gii')
        roi2gii(con_vec_med, atlasfile, output_path, f'pwISC_desc-median_fwhm-{fwhm}_confounds-{confounds_idx}_atlas-{atlas}')

    median_ISC[atlas] = con_vec_med
    pw_ISC[atlas] = _pw_ISC


n_patients = patients.sum()
n_controls = controls.sum()